# Handling errors in jobflow-remote

While it would be nice that workflows would always complete without encountering any issue, this is not always the case when running many workflows in an high-throughput regime. There can be many sources of errors during the execution of a workflow in jobflow-remote:

* user dependent: an error in the configuration, a misalignment in the versions on the code on the Runner and on the worker
* connection or transfer: since jobflow-remote needs to connect to remote works to transfer files or check jobs, errors in these actions may interrupt the workflow execution
* worker issues: any problem on the HPC system leading to a failure in the Job execution
* Job errors: errors related to the Job execution. Typically due to exceptions raised inside the Job (for a simulation it may be related to a failure in the execution of the code)

An explanation about how to deal with errors can be found in [jobflow-remote's documentation](https://matgenix.github.io/jobflow-remote/user/errors.html). Here we will consider a few examples of potential failures.

## Job errors

Let's start with a case where the Job fails due to an error during the execution. To do that we will purposely pass a wrong input, that causes the Job to raise an exception at runtime. Submit it to the local runner for simplicity. Start the Runner if not already active.

In [ ]:
from jobflow_remote.testing import add
from jobflow_remote import submit_flow

# "a" is a bad input for an addition with an integer.
j_wrong_input = add("a", 1) 

submit_flow(j_wrong_input, worker="local_shell")


Start the runner, if it is not running already, and check the status of the Job with the `jf job list` command. It should quickly turn to the `FAILED` state. 

In [ ]:
!jf job list

As suggested by the command output, the best way to start exploring the origin of the failure if with the `jf job info JOB_ID` command and check the `error` keyword. In this case the stack trace of the python exception raised during the job will be reported.

In [ ]:
!jf job info {j_wrong_input.uuid} 

In a more complex workflow, the stack trace may hint at a problem in the execution of an external code or to some other issue. In that case the best option would be to connect to the worker where the job was executed and check the output files in the `run_dir` directory. It may also be useful to check the `queue.out` and `queue.err` files produced by the queueing system (e.g. SLURM). Their content is also stored in the database and can be shown incresing the verbosity in the `jf job info` command under `remote.queue_out` and `remote.queue_err`. Keep in mind that several messages and warnings may end up in those files and not all will necessarily related to the error. In this specific case the python stack trace should be present in `queue_out`.

In [ ]:
!jf job info {j_wrong_input.uuid} -v

In general, once the source of the error has been determined, the easiest approach would be to submit a new job with the correct input and run it. It is possible to change the input values, by altering the content of the database, but this implies e detailed knowledge of how the Job has been serialized. In more involved Jobs (like those implemented in atomate2) this may not be so trivial. Implementing a general tool to do that will not be feasible, since the correct action will depend on the structure of the specific Job object, nonetheless we plan to implement some command line tool specific for the atomate2 Jobs.

Assuming that in this case the Job is going to be executed again with the corrected output, it may be useful to remove the failed workflow from the database. This can again be done through the command line interface with the `jf flow delete` command

<div class="alert alert-block alert-warning"><b>Warning:</b> while a <code>jf job delete</code> command exists, this is usually reserved for advanced use cases. In fact, arbitrarily deleting a Job may leave the workflow in an inconsistent state and should only be when properly understanding the structure of the workflow.</div>

In this case the Flow can be deleted based on the `uuid` of the job. As in the previous cases here the `-f` option is used to avoid asking for confirmation, which would not be possible inside the notebook, but it would be preferable to limit its usage.

In [ ]:
!jf flow delete -f -jid {j_wrong_input.uuid}

## Remote errors

In jobflow-remote the term "remote errors" refers to errors happening while the Runner deals with a Job. These include, for example, errors copying the file to and from a worker, or interacting with the queue manager. This means that they are typically not dependent on the Job itself, but rather on the interaction with the worker or on the options selected for the execution.

To illustrate this potential error let's consider again a simple Flow, submit it to the `local_slurm` worker, but pass a wrong value for the resources. For example using the wrong name for the SLURM partition.

In [ ]:
j_bad_resources = add(1, 2) 

resources = {"nodes": 1 , "ntasks": 1, "time": "00:10:00", "partition": "wrong_partition"}
submit_flow(j_bad_resources, worker="local_slurm", resources=resources)

Assuming that the Runner is still active, you can check the job state and see that its state will quickly turn to the <span style="color:gold">UPLOADED</span> state (note the yellow color) and after roughly 1 minute to <span style="color:red">REMOTE ERROR</span>.

In [ ]:
!jf job list

The `jf job info` command allows to confirm that the issue is indeed related to the wrong partition name.

In [ ]:
!jf job info {j_bad_resources.uuid}

What exactly happened at this Job? After uploading the Job inputs and setting the Job in the `UPLOADED` state, the Runner tried to submit the Job to the queue, but the procedure failed due to the wrong partition name. The Runner is not able to determine if the failure is related to a temporary issue with the queueing system, so set the Job up for retrying after a short while. In what is usually referred to as an exponential backoff, the Runner does not retry immediately to repeat the action, but rather retries 3 times with increasing delays after each failure. After three attempts the Job is set to the `REMOTE ERROR` state. Note that the default values for the delays are 30 seconds after the first failure, 5 minutes after the second and 30 minutes after the third one. In this project the configurations have been modified to reduce the waiting time to reache the `REMOTE ERROR` state.

### Fixing and rerunning the Job

Having determined the cause of the failure, it is possible to address this directly in the Job by changing the value of the resources to the correct one. The `jf job set resources` command can be used in this case and the effect can be verified with the `jf job info -v` command checking the `remote.resources` values:

In [ ]:
!jf job set resources -jid {j_bad_resources.uuid}:1 partition=debug

In [ ]:
!jf job info {j_bad_resources.uuid} -v

At this point there are two options and you might want to choose this carefully depending on the state at which the Job was stopped:

* **Rerun**: The Job will be brought back to the `READY` state and all the steps will be repeated. **The `run_dir` folder will also be deleted before executing the Job again**. This is useful if the failure requires the inputs and/or files to be transferred again, but will guarantee that the Job is executed from scratch.
* **Retry**: Only the step that failed will be repeated. In this specific example the state would be restored to `UPLOADED` and the Runner will attempt to submit the job again.

The choice will not make a big difference in this case, but if the failure happened due to a connection issue during the download of the outputs after the job has already been executed on the worker, rerunning will require going through the execution of the whole Job again. If instead the inputs have been modified and the Job needs to be executed, it will be necessary "rerun" it.

The most suitable action is this case is to "retry", but the "rerun" will work fine as well. With this the Job should complete correctly.

In [ ]:
!jf job retry {j_bad_resources.uuid}

In [ ]:
!jf job list

## Further checks

If the messages reported in the Job information are not enough to determine the source of the error a few more sources should be considered:
* checking the files produced on the worker
* verify that the runner processes are active
* check the runner logs in `~/.jfremote/PROJECT_NAME/log`

## Additional exercises

* Connect to the worker with a failed jobs to check if you can reach the run folder and explore the content of the files.
* Use the `add_sleep` Job from the [test Jobs available in jobflow-remote](https://github.com/Matgenix/jobflow-remote/blob/develop/src/jobflow_remote/testing/__init__.py) (sleeps for a number of seconds equivalent to the second argument provided) to submit a Job that will sleep for a long time and give a very short time limit to the SLURM `resources`. What happens? Can you fix the problem with `f job set resources`?
* Shut down the SLURM container (only the SLURM one!) to mimick an issue in the connection with the worker and try submitting a new Job. After it fails restart the container and use the `jf job retry` command.
* Implement a Job here in the notebook, create a Flow with it and submit it.
  
  ```python
  from jobflow import job

  @job
  def example_job(a,b):
      pass

  submit_flow(example_job(1,2), worker="local_slurm")
  ```
  
  Will it run?
* Create a new `j_bad_resources` and submit it, while the job is in the <span style="color:gold">UPLOADED</span> state (yellow color -> failure), use `jf job list -vvv`. Can you see when it will be retried again?